In [1]:
# things we need for NLP
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

# things we need for Tensorflow
import numpy as np
import tflearn
import tensorflow as tf
import random

Instructions for updating:
non-resource variables are not supported in the long term
curses is not supported on this machine (please install/reinstall curses for an optimal experience)



We’ll build a simple state-machine to handle responses, using our intents model (from the previous step) as our classifier. [That’s how chatbots work](https://medium.freecodecamp.com/how-chat-bots-work-dfff656a35e2).

> A contextual chatbot framework is a classifier within a state-machine.

- we’ll un-pickle our model and documents as well as reload our intents file
-  Remember our chatbot framework is separate from our model build — you don’t need to rebuild your model unless the intent patterns change
- With several hundred intents and thousands of patterns the model could take several minutes to build

In [2]:
# restore all of our data structures
import pickle
data = pickle.load(open("training_data", "rb"))
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']

# import our chat-bot intents file
import json
with open('Mop intents.json') as json_data:
    intents = json.load(json_data)

In [ ]:
"""""input = keras.input(shape=[len(intent_train_x[0])]) 
intent_net = input
intent_net = layers.Dense(8)(intent_net) 
intent_net = layers.Dense(8)(intent_net) 
intent_net = layers.Dense(len(intent_train_y[0], activation='softmax')(intent_net) 
intent_model = keras.Model(inputs=[input], outputs=[output])
intent_model.load('model1') """""

In [6]:
# Build neural network
net = tflearn.input_data(shape=[None, len(train_x[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
net = tflearn.regression(net)

# Define model and setup tensorboard
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Util functions

In [7]:
def clean_up_sentence(sentence):
    # tokenize the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence


def bow(sentence, words, show_details=False):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words
    bag = [0]*len(words)
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print("found in bag: %s" % w)

    return(np.array(bag))

In [8]:
p1 = bow("what is mopedo?", words)
print (p1)
print (classes)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0]
['', 'Cities', 'Distinct features', 'Estimated time of arrival', 'Guardian booking', 'Modes of Payment', 'Mopedo', 'Per ride', 'Price Estimation', 'Refer Friend', 'Registration', 'Ride Insurance', 'Ride Service', 'Ride history', 'Safety', 'Saving option', 'Services', 'Tracking Ride', 'Women pilots', 'cancellation', 'convenient for Women', 'distance limit', 'goodbye', 'greeting', 'payments', 'pilot details', 'safety precautions', 'schedule', 'thanks', 'waiting charges']


In [9]:
# load our saved model
model.load('./model.tflearn')

INFO:tensorflow:Restoring parameters from C:\Users\POOJA\Mopybot\model.tflearn








Checking that our model gives the same result as in the previous step

In [11]:
def get_predicted_intent(predictions):
    return classes[np.argmax(predictions)]


assert 'Mopedo' == get_predicted_intent(model.predict([p1]))

## Response processor

In [12]:

# data structure to hold user context
context = {}

ERROR_THRESHOLD = 0.25


def classify(sentence):
    # generate probabilities from the model
    results = model.predict([bow(sentence, words)])[0]

    # filter out predictions below a threshold
    results = [[i, r] for i, r in enumerate(results) if r > ERROR_THRESHOLD]

    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []

    for r in results:
        return_list.append((classes[r[0]], r[1]))

    # return tuple of intent and probability
    return return_list


def response(sentence, userID='123', show_details=False):
    results = classify(sentence)
    if results:
        for i in intents['intents']:
            if i['tag'] == results[0][0]:
                result = (random.choice(i['responses']))
                return result
    else:
        return ("info@")
        

In [13]:
classify("Mopedo")

[('Mopedo', 0.99640083)]

In [14]:
response('Mopedo')

'Mopedo is a Bike-Taxi hailing application integrated with advanced AI technology offering the most affordable and reliable commute service to every common man by considering every second of customer intent.'

In [15]:
response('what services does mopedo provide?')

'The application provides Mopedo Bike and Auto services.'

In [ ]:
classify("safety and maintenance")

In [16]:
response('How safe is mopedo?')

'The application ensures high-end security for users with safe and home free rides.'

In [17]:
response('payments?')

'Mopedo allows cash payment and also accepts credit/debit cards, digital wallets(gpay, phonepe, paytm) which saves from hassle of tendering the exact change.'

In [20]:
response('can I book ride for others?')

'Yes, Of course.! Anyone with the mopedo application can book a ride for themselves and guardians just by specifying the guardianâ€™s details.'

In [21]:
response('Mopedo city boundaries?')

'It operates across major cities in India. These areas will change overtime as we grow our presence across the country.'

In [22]:
response("how are they charged?")

'Mopedo pricing depends upon Distance travelled, Minimum fare, Per km and Per min which vary based on the pickup and destination locations.'

In [ ]:
response("Any technical help")

In [ ]:
response(" help")

In [ ]:
response("..")

In [ ]:
response("ha")

In [ ]:
response("puja")

In [ ]:
response(".")